# ✅ **Answer Correctness**

   - **What It Measures**: This is about straight-up accuracy – how well does the answer align with the ground truth?

   - **Scoring**: Judged on a 0 to 1 scale, where higher scores signal a bullseye match with the ground truth.

   - **Example**:

     - 🟢 Ground Truth: Photosynthesis in plants primarily occurs in the chloroplasts.

     - ⬆️ High answer correctness: Photosynthesis takes place in the chloroplasts of plant cells.

     - ⬇️ Low answer correctness: Photosynthesis occurs in the mitochondria of plants.